In [1]:
import os

documents = []

for (root, dirs, files) in os.walk('../documents'):
    for file in files:
        fp = f'{root}/{file}'
        with open(fp, 'r', encoding='utf-8') as f:
            content = f.read()
        
        doc = {
            'name': file.replace('.md', ''),
            'content': content
        }

        documents.append(doc)

print(len(documents))

94


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

In [3]:
char_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ", ", " ", ""],
    chunk_size=1000,
    chunk_overlap=10
)
char_splitter_texts = char_splitter.split_text('\n\n'.join([d['content'] for d in documents]))

print(len(char_splitter_texts))

114


In [4]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=10, tokens_per_chunk=256)

token_splitter_texts = []
for text in char_splitter_texts:
    token_splitter_texts += token_splitter.split_text(text)

print(len(token_splitter_texts))

/home/lisboa/miniconda3/envs/promptneering/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


115


In [5]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()

In [ ]:
client = chromadb.PersistentClient(path='../chroma')
collection = client.get_or_create_collection("prompt_engineering_knowledge", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_splitter_texts))]

# collection.add(ids=ids, documents=token_splitter_texts)

print(collection.count())

115


In [7]:
results = collection.query(query_texts=['How to make a few show prompting?'], n_results=10)

In [8]:
results['documents']

[[', including " prompt - oirl, " " opro, " " autoprompt, " " prefix tuning, " and " prompt tuning, " which explore various methods for improving prompt generation and optimization.',
  'few - shot prompting enhances the performance of large language models on complex tasks by providing demonstrations in the prompt, allowing for in - context learning. this technique was highlighted in an example from brown et al. 2020, where the model successfully used the word " farduddle " in a sentence after being given a single example. increasing the number of demonstrations can improve results, as shown in various experiments. however, few - shot prompting has limitations, particularly with complex reasoning tasks, as illustrated by a failed attempt to determine if a set of odd numbers summed to an even number. despite providing multiple examples, the model still produced an incorrect answer, indicating the need for more advanced prompt engineering techniques, such as chain - of - thought prompti